In [1]:
## Connect to gg driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AI Tutor/CV/Topic2: Object Classification/day06

/content/drive/.shortcut-targets-by-id/1g_hBCGxmI5lTFXyvD-igJcroxCjDzObt/AI Tutor/CV/Topic2: Object Classification/day06


In [11]:
%%capture
!pip install onnxruntime
!pip install onnxruntime-gpu
!pip install netron
!pip install onnx
!pip install pyngrok

### Common function usage


In [12]:
import torch
import torch.onnx
import onnxruntime
import numpy as np

In [13]:
def convert_pytorch2onnx(model, input_samples, path_onnx, mode = 'float32bit', device = 'cuda'):
    if mode == 'float16bit':
        print("float16bit")
        model.float()
        model.half()  # Chuyển mô hình sang float16
        input_samples = input_samples.half()
    model.to(device)
    model.eval()
    input_samples = input_samples.to(device)
    torch.onnx.export(
        model, # Model load bằng pytorch
        input_samples, # input với kích thước mong muốn
        path_onnx, # path_onnx bạn lưu
        verbose=False,  # Hiển thị thông báo trong quá trình chuyển đổi
        opset_version=12, # Đọc xem mỗi version sẽ có hỗ trợ onnx cho lớp nào
        do_constant_folding=True , # Dùng Constant-folding optimizer giúp cải thiện tốc độ và dung lượng
        input_names = ['images'],   # the model's input names
        output_names = ['output'], # the model's output names
        dynamic_axes={
                'images' : {0 : 'batch_size',},
                'output' : {0 : 'batch_size'}
            }
        )

def load_onnx_model(path_onnx, providers=['CUDAExecutionProvider', 'CPUExecutionProvider']):
    # Create an ONNX Runtime inference session for the ONNX model
    ort_session = onnxruntime.InferenceSession(
        path_onnx,
        providers=providers
        )
    return ort_session

def onnx_infer(ort_session, input_data):
    ort_inputs = {ort_session.get_inputs()[0].name: input_data}
    ort_output = ort_session.run(None, ort_inputs)
    return ort_output

## Convert model to onnx

In [14]:
## Load model pytorch
model = torch.load("models/torch/resnet18.pth")

In [15]:
mode ='float32bit'
path_onnx = f'models/onnx/vgg19_{mode}.onnx'
input_samples = torch.randn(16, 3, 224, 224)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
convert_pytorch2onnx(
    model=model,
    input_samples=input_samples,
    path_onnx=path_onnx,
    mode=mode,
    device=device
)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [16]:
mode ='float16bit'
path_onnx = f'models/onnx/vgg19_{mode}.onnx'
input_samples = torch.randn(16, 3, 224, 224)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
convert_pytorch2onnx(
    model=model,
    input_samples=input_samples,
    path_onnx=path_onnx,
    mode=mode,
    device=device
)


float16bit
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [17]:
batch_size = 4
input_data = torch.randn(batch_size, 3, 224, 224)


In [18]:
onnx_float16 = load_onnx_model('models/onnx/vgg19_float16bit.onnx')
input_16bit = input_data.half()
input_numpy = input_16bit.numpy()
onnx_infer(onnx_float16, input_numpy)

[array([[-5.176 ,  0.298 ,  0.756 , -5.61  ,  6.58  , -1.377 ],
        [-4.418 ,  0.1497,  0.5835, -5.36  ,  5.918 , -1.191 ],
        [-5.39  ,  0.45  ,  0.746 , -5.69  ,  6.71  , -1.471 ],
        [-4.934 ,  0.2207,  0.6846, -5.746 ,  6.23  , -1.006 ]],
       dtype=float16)]

In [19]:
onnx_float32 = load_onnx_model('models/onnx/vgg19_float32bit.onnx')
input_numpy = input_data.numpy()
onnx_infer(onnx_float32, input_numpy)

[array([[-5.1684933 ,  0.30329192,  0.75374323, -5.5970097 ,  6.568671  ,
         -1.3846576 ],
        [-4.411299  ,  0.15472548,  0.581463  , -5.348566  ,  5.911161  ,
         -1.1980268 ],
        [-5.3860726 ,  0.4554967 ,  0.7447722 , -5.6803846 ,  6.7021537 ,
         -1.478588  ],
        [-4.9252305 ,  0.2271531 ,  0.68288165, -5.733523  ,  6.221128  ,
         -1.0147607 ]], dtype=float32)]

In [21]:
## Load model pytorch
model = torch.load("models/torch/resnet18.pth")
model.to('cuda')
with torch.no_grad():
    out = model(input_data.to('cuda'))
out

tensor([[-5.1685,  0.3033,  0.7537, -5.5970,  6.5687, -1.3847],
        [-4.4113,  0.1547,  0.5815, -5.3486,  5.9112, -1.1980],
        [-5.3861,  0.4555,  0.7448, -5.6804,  6.7022, -1.4786],
        [-4.9252,  0.2272,  0.6829, -5.7335,  6.2211, -1.0148]],
       device='cuda:0')

In [22]:
#!pip install netron
import netron
netron.start('models/onnx/vgg19_float32bit.onnx', 8081)

Serving 'models/onnx/vgg19_float32bit.onnx' at http://localhost:8081


('localhost', 8081)

In [35]:
# https://netron.app/
# chạy online lấy weight onnx upload lên